In [1]:
import torch
import pandas as pd
import numpy as np
import glob
import os
import gc
from torch_geometric.data import HeteroData
from sklearn.preprocessing import LabelEncoder

# --- CẤU HÌNH ---
input_dir = r"C:\Users\dotie\OneDrive\Documents\FAERS\Processed_Chunks"
all_files = glob.glob(os.path.join(input_dir, "processed_final_*.csv"))

# --- 1. XÂY DỰNG TỪ ĐIỂN ID (MAPPING) ---
# Để tiết kiệm RAM, ta chỉ quét tên cột (vốn chứa tên thuốc/bệnh) chứ không đọc toàn bộ dữ liệu
print("--- Bước 1: Quét tên thuốc và phản ứng để tạo ID ---")

# Đọc file đầu tiên để lấy danh sách cột (Vì các file đã đồng bộ cột)
df_cols = pd.read_csv(all_files[0], nrows=0) 
drug_names = [c for c in df_cols.columns if c.startswith('x ') and c not in ['x Age', 'x Gender', 'x AgeGroup']]
reaction_names = [c for c in df_cols.columns if c.startswith('y ')]

print(f"Số lượng thuốc: {len(drug_names)}")
print(f"Số lượng phản ứng: {len(reaction_names)}")

# --- 2. XÂY DỰNG DANH SÁCH CẠNH (EDGE LIST) ---
print("--- Bước 2: Đọc dữ liệu và tạo danh sách cạnh ---")

# Khởi tạo list chứa cạnh
src_patient_drug = []
dst_patient_drug = []
src_patient_react = []
dst_patient_react = []

patient_features_list = []
global_patient_id = 0 # ID bệnh nhân tăng dần

for f in all_files:
    # Đọc từng chunk
    print(f"Processing: {os.path.basename(f)}")
    df = pd.read_csv(f)
    
    num_rows = len(df)
    local_patient_ids = np.arange(num_rows) + global_patient_id
    
    # 1. Lấy đặc trưng bệnh nhân (Age, Gender)
    # Normalize Age: chia 100
    p_feats = df[['x Age', 'x Gender']].values
    p_feats[:, 0] = p_feats[:, 0] / 100.0 
    patient_features_list.append(p_feats)
    
    # 2. Tạo cạnh Patient -> Drug
    # Lấy ma trận con của thuốc (dạng dense)
    # Lưu ý: Nếu máy yếu, đoạn này có thể làm từng dòng, nhưng numpy vectorization nhanh hơn
    drug_matrix = df[drug_names].values
    # Tìm các vị trí thuốc > 0
    row_indices, col_indices = np.where(drug_matrix > 0)
    
    # Map row index local -> global patient ID
    src_patient_drug.append(local_patient_ids[row_indices])
    dst_patient_drug.append(col_indices) # col_indices chính là ID của thuốc (do thứ tự cột cố định)
    
    # 3. Tạo cạnh Patient -> Reaction
    react_matrix = df[reaction_names].values
    row_indices_r, col_indices_r = np.where(react_matrix > 0)
    
    src_patient_react.append(local_patient_ids[row_indices_r])
    dst_patient_react.append(col_indices_r)
    
    global_patient_id += num_rows
    
    # Dọn RAM
    del df, drug_matrix, react_matrix
    gc.collect()

# Gộp các list lại thành numpy array lớn
print("Đang ghép nối dữ liệu...")
patient_features = np.vstack(patient_features_list)
edge_index_drug = np.vstack([np.concatenate(src_patient_drug), np.concatenate(dst_patient_drug)])
edge_index_react = np.vstack([np.concatenate(src_patient_react), np.concatenate(dst_patient_react)])

# --- 3. TẠO HETERODATA ---
print("--- Bước 3: Tạo Graph Object ---")
data = HeteroData()

# Nodes
data['patient'].x = torch.from_numpy(patient_features).float()
data['patient'].num_nodes = global_patient_id
# Drug & Reaction không cần features input, ta sẽ dùng Embedding Layer trong model
data['drug'].num_nodes = len(drug_names)
data['reaction'].num_nodes = len(reaction_names)

# Edges (Chuyển sang LongTensor)
data['patient', 'takes', 'drug'].edge_index = torch.from_numpy(edge_index_drug).long()
data['patient', 'has_reaction', 'reaction'].edge_index = torch.from_numpy(edge_index_react).long()

# Tạo cạnh ngược (Reverse edges) cho GNN
data['drug', 'taken_by', 'patient'].edge_index = torch.flip(data['patient', 'takes', 'drug'].edge_index, [0])
data['reaction', 'reaction_in', 'patient'].edge_index = torch.flip(data['patient', 'has_reaction', 'reaction'].edge_index, [0])

print("Thông tin đồ thị:", data)

# Lưu xuống ổ cứng để dùng cho file train
torch.save(data, "hetero_graph_data.pt")
print("✅ Đã lưu file 'hetero_graph_data.pt'. Xong phần chuẩn bị!")

c:\Users\dotie\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- Bước 1: Quét tên thuốc và phản ứng để tạo ID ---
Số lượng thuốc: 5259
Số lượng phản ứng: 10488
--- Bước 2: Đọc dữ liệu và tạo danh sách cạnh ---
Processing: processed_final_1.csv
Processing: processed_final_10.csv
Processing: processed_final_100.csv
Processing: processed_final_101.csv
Processing: processed_final_102.csv
Processing: processed_final_103.csv
Processing: processed_final_104.csv
Processing: processed_final_105.csv
Processing: processed_final_106.csv
Processing: processed_final_107.csv
Processing: processed_final_108.csv
Processing: processed_final_109.csv
Processing: processed_final_11.csv
Processing: processed_final_110.csv
Processing: processed_final_111.csv
Processing: processed_final_112.csv
Processing: processed_final_113.csv
Processing: processed_final_114.csv
Processing: processed_final_115.csv
Processing: processed_final_116.csv
Processing: processed_final_117.csv
Processing: processed_final_118.csv
Processing: processed_final_119.csv
Processing: processed_final

In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import glob
import os
import gc
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, to_hetero, Linear

# ---------------------------------------------------------
# 1. CẤU HÌNH & CHUẨN BỊ
# ---------------------------------------------------------
input_dir = r"C:\Users\dotie\OneDrive\Documents\FAERS\Processed_Chunks"
all_files = glob.glob(os.path.join(input_dir, "processed_final_*.csv"))

print(f"Tìm thấy {len(all_files)} file dữ liệu.")

# Đọc file đầu tiên để lấy thông tin kích thước (Số lượng thuốc/phản ứng toàn cục)
df_temp = pd.read_csv(all_files[0], nrows=1)
drug_cols = [c for c in df_temp.columns if c.startswith('x ') and c not in ['x Age', 'x Gender', 'x AgeGroup']]
reaction_cols = [c for c in df_temp.columns if c.startswith('y ')]
num_drugs = len(drug_cols)
num_reactions = len(reaction_cols)

print(f"Số lượng thuốc (Input Nodes): {num_drugs}")
print(f"Số lượng phản ứng (Output Nodes): {num_reactions}")

# ---------------------------------------------------------
# 2. ĐỊNH NGHĨA MODEL
# ---------------------------------------------------------
class HeteroSage(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers=2):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = SAGEConv((-1, -1), hidden_channels)
            self.convs.append(conv)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index).relu()
        return x

# Metadata giả lập để khởi tạo model (cấu trúc đồ thị)
# Ta cần khai báo các loại node và edge sẽ xuất hiện
metadata = (
    ['patient', 'drug', 'reaction'],
    [('patient', 'takes', 'drug'), 
     ('drug', 'taken_by', 'patient'),
     ('patient', 'has_reaction', 'reaction'),
     ('reaction', 'reaction_in', 'patient')]
)

model = HeteroSage(hidden_channels=64, out_channels=64, num_layers=2)
model = to_hetero(model, metadata, aggr='sum')

# Embedding layers cho Drug và Reaction
# +1 để dự phòng index
drug_emb = torch.nn.Embedding(num_drugs + 1, 64)
react_emb = torch.nn.Embedding(num_reactions + 1, 64)
# Linear layer cho Patient (Age, Gender)
patient_lin = Linear(2, 64)

optimizer = torch.optim.Adam(
    list(model.parameters()) + list(drug_emb.parameters()) + list(react_emb.parameters()) + list(patient_lin.parameters()), 
    lr=0.01
)

# ---------------------------------------------------------
# 3. HÀM CHUYỂN ĐỔI DATAFRAME -> GRAPH (Mini-Graph)
# ---------------------------------------------------------
def create_mini_graph(df):
    # 1. Patient Features
    p_feats = df[['x Age', 'x Gender']].values
    p_feats[:, 0] = p_feats[:, 0] / 100.0 
    x_patient = torch.tensor(p_feats, dtype=torch.float)
    
    # 2. Edges: Patient -> Drug
    # Dùng numpy where để tìm các ô có giá trị > 0 (người dùng thuốc)
    # drug_cols đã được sort và cố định vị trí ở bước chuẩn bị
    drug_matrix = df[drug_cols].values
    rows, cols = np.where(drug_matrix > 0)
    edge_index_drug = torch.tensor([rows, cols], dtype=torch.long)
    
    # 3. Edges: Patient -> Reaction
    react_matrix = df[reaction_cols].values
    rows_r, cols_r = np.where(react_matrix > 0)
    edge_index_react = torch.tensor([rows_r, cols_r], dtype=torch.long)
    
    # Tạo HeteroData
    data = HeteroData()
    data['patient'].x = x_patient
    data['patient'].num_nodes = len(df)
    
    # Gán Embedding (ID) cho thuốc/phản ứng
    # Ở đây ta truyền vào indices từ 0 -> num_drugs để Embedding layer tra cứu
    data['drug'].x = torch.arange(num_drugs)
    data['reaction'].x = torch.arange(num_reactions)
    
    data['patient', 'takes', 'drug'].edge_index = edge_index_drug
    data['patient', 'has_reaction', 'reaction'].edge_index = edge_index_react
    
    # Cạnh ngược
    data['drug', 'taken_by', 'patient'].edge_index = torch.flip(edge_index_drug, [0])
    data['reaction', 'reaction_in', 'patient'].edge_index = torch.flip(edge_index_react, [0])
    
    return data

# ---------------------------------------------------------
# 4. TRAINING LOOP (Chunk-based)
# ---------------------------------------------------------
print("Bắt đầu training (File-by-File)...")

def train_one_epoch():
    model.train()
    total_loss = 0
    total_examples = 0
    
    # Biến để tính metrics toàn cục cho Epoch
    total_tp = 0 # True Positives
    total_fp = 0 # False Positives
    total_fn = 0 # False Negatives
    
    # Duyệt qua từng file CSV
    for i, f_path in enumerate(all_files):
        try:
            df_chunk = pd.read_csv(f_path)
        except:
            continue
            
        batch = create_mini_graph(df_chunk)
        optimizer.zero_grad()
        
        # --- Forward Pass ---
        x_dict = {
            'patient': patient_lin(batch['patient'].x),
            'drug': drug_emb(batch['drug'].x),
            'reaction': react_emb(batch['reaction'].x)
        }
        
        out = model(x_dict, batch.edge_index_dict)
        
        edge_index = batch['patient', 'has_reaction', 'reaction'].edge_index
        if edge_index.numel() == 0: continue

        src_emb = out['patient'][edge_index[0]]
        dst_emb = out['reaction'][edge_index[1]]
        
        # --- Tính điểm (Logits) ---
        pos_score = (src_emb * dst_emb).sum(dim=-1) # Logits cho mẫu Dương (Có thật)
        
        neg_dst_idx = torch.randint(0, num_reactions, (len(pos_score),))
        neg_dst_emb = out['reaction'][neg_dst_idx]
        neg_score = (src_emb * neg_dst_emb).sum(dim=-1) # Logits cho mẫu Âm (Giả)
        
        # --- Tính Loss ---
        scores = torch.cat([pos_score, neg_score])
        labels = torch.cat([torch.ones_like(pos_score), torch.zeros_like(neg_score)])
        
        loss = F.binary_cross_entropy_with_logits(scores, labels)
        loss.backward()
        optimizer.step()
        
        # --- TÍNH TOÁN METRICS CHO BATCH NÀY ---
        # Ngưỡng phân loại là 0 (vì Sigmoid(0) = 0.5)
        # Sử dụng torch.no_grad() để không ảnh hưởng đến gradient
        with torch.no_grad():
            # TP: Mẫu dương có điểm dự đoán > 0
            tp = (pos_score > 0).sum().item()
            # FN: Mẫu dương có điểm dự đoán <= 0
            fn = (pos_score <= 0).sum().item()
            # FP: Mẫu âm có điểm dự đoán > 0
            fp = (neg_score > 0).sum().item()
            
            total_tp += tp
            total_fp += fp
            total_fn += fn
        
        batch_size = len(df_chunk)
        total_loss += loss.item() * batch_size
        total_examples += batch_size
        
        del df_chunk, batch
        
        if i % 10 == 0:
            print(f"  Processed {i}/{len(all_files)} files. Loss: {loss.item():.4f}")

    # --- TÍNH METRICS TỔNG HỢP CUỐI EPOCH ---
    epsilon = 1e-9 # Số nhỏ để tránh lỗi chia cho 0
    
    # 1. Precision = TP / (TP + FP)
    # (Trong số những cái máy đoán là có phản ứng, bao nhiêu % là đúng?)
    precision = total_tp / (total_tp + total_fp + epsilon)
    
    # 2. Recall = TP / (TP + FN)
    # (Trong số các phản ứng thực tế, máy bắt được bao nhiêu %?)
    recall = total_tp / (total_tp + total_fn + epsilon)
    
    # 3. F1-Score = 2 * (P * R) / (P + R)
    f1 = 2 * (precision * recall) / (precision + recall + epsilon)
    
    avg_loss = total_loss / (total_examples + epsilon)
    
    return avg_loss, precision, recall, f1

    # Chạy Training
for epoch in range(1, 2): # Ví dụ chạy 10 Epochs
    print(f"\n================ EPOCH {epoch} ================")
    
    # Gọi hàm train và nhận về 4 giá trị
    avg_loss, prec, rec, f1 = train_one_epoch()
    
    print(f"✅ KẾT THÚC EPOCH {epoch}")
    print(f"   - Average Loss: {avg_loss:.4f}")
    print(f"   - Precision:    {prec:.4f}")
    print(f"   - Recall:       {rec:.4f}")
    print(f"   - F1-Score:     {f1:.4f}")

print("\n🎉 Đã train xong! (Model saved)")

# ---------------------------------------------------------
# 5. LƯU MODEL & EMBEDDINGS
# ---------------------------------------------------------
torch.save(model.state_dict(), "HGNN_Model.pth")
torch.save(drug_emb.state_dict(), "Drug_Embeddings.pth")
torch.save(react_emb.state_dict(), "Reaction_Embeddings.pth")
torch.save(patient_lin.state_dict(), "Patient_Encoder.pth")
# Lưu danh sách tên thuốc/phản ứng để map lại sau này
import joblib
joblib.dump(drug_cols, "Drug_Names.pkl")
joblib.dump(reaction_cols, "Reaction_Names.pkl")

c:\Users\dotie\AppData\Local\Programs\Python\Python314\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tìm thấy 133 file dữ liệu.
Số lượng thuốc (Input Nodes): 5259
Số lượng phản ứng (Output Nodes): 10488
Bắt đầu training (File-by-File)...

================ EPOCH 1 ================


C:\Users\dotie\AppData\Local\Temp\ipykernel_8496\1847126495.py:84: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  edge_index_drug = torch.tensor([rows, cols], dtype=torch.long)


  Processed 0/133 files. Loss: 0.8604
  Processed 10/133 files. Loss: 0.4947
  Processed 20/133 files. Loss: 0.4974
  Processed 30/133 files. Loss: 0.5369
  Processed 40/133 files. Loss: 0.4246
  Processed 50/133 files. Loss: 0.4147
  Processed 60/133 files. Loss: 0.3669
  Processed 70/133 files. Loss: 0.4295
  Processed 80/133 files. Loss: 0.4584
  Processed 90/133 files. Loss: 0.4431
  Processed 100/133 files. Loss: 0.3799
  Processed 110/133 files. Loss: 0.4427
  Processed 120/133 files. Loss: 0.4152
  Processed 130/133 files. Loss: 0.4122
✅ KẾT THÚC EPOCH 1
   - Average Loss: 0.4446
   - Precision:    0.9187
   - Recall:       0.9732
   - F1-Score:     0.9451

🎉 Đã train xong! (Model saved)


['Reaction_Names.pkl']